In [4]:
import quantumpropagator as qp
import matplotlib.pyplot as plt
from ipywidgets import interact,fixed #, interactive, fixed, interact_manual
import ipywidgets as widgets
import os
import glob
#%matplotlib notebook
%matplotlib ipympl

plt.rcParams.update({'font.size': 10})

In [7]:
fol = '/home/alessio/Desktop/Dropbox/USA/bet/'
subfolders = sorted([dir for dir in os.listdir(fol) if os.path.isdir(os.path.join(fol,dir)) and dir != os.path.join(fol,'.ipynb_checkpoints')])
print(''.join(['{} -> {}\n'.format(a,b) for a,b in enumerate(subfolders)]))

0 -> b-pulseAlongX_0.22_goodG_0000
1 -> b-pulseAlongX_0.22_short_0_goodG_0000
2 -> b-pulseAlongX_0.22_short_pi_goodG_0000
3 -> c-Experiments_on_Nac_0000
4 -> d-experiments_smooth01_0000
5 -> d-experiments_smooth10_0000
6 -> d-experiments_smooth50_0000
7 -> h-IR-X_80_0000
8 -> i-IR-Z_80_0000
9 -> k-IR-Y_80_0000
10 -> l-IR-XZ-50_0000
11 -> l-IR-XZ-70_0000
12 -> l-IR-XZ-90_0000
13 -> m-only_IR_longer_with_nac_2_1_0000
14 -> m-only_IR_longer_with_nac_2_1_counterClock_0000
15 -> m-only_IR_longer_with_nac_2_1_inverted_phase_0000
16 -> m-only_IR_longer_with_nac_2_1_phase_pi_0000
17 -> m-only_IR_middle100_0000
18 -> m-only_IR_middle140_0000
19 -> m-only_IR_middle50_0000
20 -> m-only_IR_middle80_0000
21 -> m-only_IR_middle90_0000
22 -> m-only_IR_retarded75_0000
23 -> m-only_IR_retarded80_0000
24 -> m-only_IR_retarded85_0000
25 -> m-only_IR_retarded90_0000
26 -> m-only_IR_short_with_nac_2_1_0000
27 -> m-only_IR_short_with_nac_2_1_inverted_phase_0000
28 -> m-only_IR_short_with_nac_2_1_phase_pi_00

In [10]:
i_want = [0]
selected = [ subfolders[i] for i in i_want]

In [41]:
def graph_between_bonds(folder_list):
    fig, ax0 = plt.subplots(1,1,figsize=(8,5))
    for this_folder in folder_list:
        abs_path = os.path.join(fol,this_folder)
        fir_file = os.path.join(abs_path, 'list_first.dat')
        sec_file = os.path.join(abs_path, 'list_second.dat')
        fir = np.loadtxt(fir_file)
        sec = np.loadtxt(sec_file)
        norm = 0.317460*0.317460*0.317460
        ax0.plot(fir[:,0],fir[:,1]*norm, color='b', label='single bond')
        ax0.plot(sec[:,0],sec[:,1]*norm, color='r', label='double bond')
    
    ax0.legend()
    ax0.set_title(this_folder)
    ax0.set_xlabel('fs')
    ax0.set_ylabel('Fraction of 50 electrons')
    
    fig.tight_layout()
        
graph_between_bonds(selected)

/home/alessio/config/miniconda/envs/quantumpropagator/lib/python3.5/site-packages/matplotlib/pyplot.py:523: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


A Jupyter Widget